In [1]:
test = '''
contract get_fib_n(n : int){

    let x: int = 0;

    entry sum(n: int){
        let a: int = 2;
        let b: int = 3;
        x = a + b;
    }
}
'''

In [2]:
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast

lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(test))
tokens = process_lexer_tokens(lexer_tokens)

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

0 	 <program> -> .contractid(<param-list>){<stat_list>}, 
	 S' -> .<program>,  

1 	 <program> -> contract.id(<param-list>){<stat_list>},  

2 	 <program> -> contractid.(<param-list>){<stat_list>},  

3 	 <param-list> -> .<param>, 
	 <program> -> contractid(.<param-list>){<stat_list>}, 
	 <param-list> -> .<param>,<param-list>, 
	 <param> -> .id:type,  

4 	 <program> -> contractid(<param-list>.){<stat_list>},  

5 	 <program> -> contractid(<param-list>).{<stat_list>},  

6 	 <stat> -> .<else-stat>, 
	 <else-stat> -> .else{<stat_list>}, 
	 <stat> -> .<def-entry>, 
	 <program> -> contractid(<param-list>){.<stat_list>}, 
	 <def-entry> -> .entryid(<param-list>){<stat_list>}, 
	 <return-stat> -> .return<expr>;, 
	 <stat> -> .<def-func>, 
	 <stat_list> -> .<stat_list><stat>, 
	 <stat> -> .<let-var>>, 
	 <var-call> -> .id=<expr>;, 
	 <stat> -> .<return-stat>, 
	 <def-func> -> .funcid(<param-list>):type{<stat_list>}, 
	 <stat> -> .<while-stat>, 
	 <stat> -> .<if-stat>, 
	 <let-var>> -> .letid:

In [3]:
productions, operations = derivation


In [4]:
ast = build_slr_ast(productions, operations, tokens)

In [5]:
from visitors.string_rep_visitor import FormatVisitor
format = FormatVisitor()
format.visit(ast)

'\\__ProgramNode: contract node.idx(n : int) [<stat>; ... <stat>;]\n\t\\__AttrDeclarationNode: n : int\n\t\t\\__VarDeclarationNode: let x = <expr> : int\n\t\t\t\\__ ConstantNumNode: 0\n\t\t\\__EntryDeclarationNode: Entry sum(n : int)\n\t\t\t\\__AttrDeclarationNode: n : int\n\t\t\t\t\\__VarDeclarationNode: let a = <expr> : int\n\t\t\t\t\t\\__ ConstantNumNode: 2\n\t\t\t\t\\__VarDeclarationNode: let b = <expr> : int\n\t\t\t\t\t\\__ ConstantNumNode: 3\n\t\t\t\t\\__VarCallNode: x = <expr>\n\t\t\t\t\t\\__ PlusNode\n\t\t\t\t\t\t\\__ VariableNode: a\n\t\t\t\t\t\t\\__ VariableNode: b'

In [6]:
from visitors.high_level_ir_generator_visitor import TzScriptToHighLevelIrVisitor
high_level_ir = TzScriptToHighLevelIrVisitor()

In [7]:
from visitors.hl_string_repre import HLReprVisitor
hl_repr = HLReprVisitor()
ir = high_level_ir.visit(ast)

mirame
mirame
statements in entry:  <parser.tzscript_ast.VarDeclarationNode object at 0x7faac97ce5d0>
statements in entry:  <parser.tzscript_ast.VarDeclarationNode object at 0x7faac97cda10>
statements in entry:  <parser.tzscript_ast.VarCallNode object at 0x7faac9713d90>
here
nodos donde suma


In [8]:
print(hl_repr.visit(ir))

los:  [<intermediate_ast.high_level_ir_ast.IfEntryNode object at 0x7faac9580190>]
If entry: [<intermediate_ast.high_level_ir_ast.PushValueNode object at 0x7faac9580110>, <intermediate_ast.high_level_ir_ast.PushVariableNode object at 0x7faac9581450>, <intermediate_ast.high_level_ir_ast.PushValueNode object at 0x7faac97e1d50>, <intermediate_ast.high_level_ir_ast.PushVariableNode object at 0x7faac9580050>, <intermediate_ast.high_level_ir_ast.GetToTopNode object at 0x7faac95808d0>, <intermediate_ast.high_level_ir_ast.GetToTopNode object at 0x7faac97d71d0>, <intermediate_ast.high_level_ir_ast.PlusNode object at 0x7faac96f49d0>, <intermediate_ast.high_level_ir_ast.PushVariableNode object at 0x7faac9696510>]
ans de variable int a
ans de variable int b
operacion
ans deplus 			\__ PlusNode
ans de variable  x
contract { entrypoint{...} storage{...} code{...}} 
	entrypoint  { <id,params> , <id,params> , ... , <id,params> }
		\__EntryPointDeclarationNode: sum ([<parser.tzscript_ast.AttrDeclaration

In [9]:
for x in ir.entrypoints.entrypoint_list:
    print(x.id)

sum
